<a href="https://colab.research.google.com/github/gigabitamin/InnEats/blob/main/model_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Jul 29 03:24:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# !pip install -q -U torch transformers datasets accelerate peft bitsandbytes trl
# !pip install -q huggingface_hub sentencepiece protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 126.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [3]:
# !pip install torch==2.6.0 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124 fastai==2.7.19 --extra-index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 725.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install -q -U transformers datasets accelerate peft bitsandbytes trl
!pip install -q huggingface_hub sentencepiece protobuf

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
from datasets import load_dataset

# 파일명이 실제 업로드한 파일명과 동일해야 합니다!
dataset = load_dataset("json", data_files="hearth_chat_instruction.jsonl", split="train")
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

{'instruction': '간단한 자기소개를 해줘', 'input': '', 'output': '안녕하세요! 저는 Hearth Chat의 AI 챗봇입니다.', 'conversations': None, 'system_prompt': None, 'question': None, 'answer': None, 'text': None, 'label': None}


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# 4bit 양자화(QLoRA)로 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# LoRA 설정
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, peft_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [10]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
# from peft import get_peft_model, LoraConfig, TaskType
# import time

# model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# # 실행 시간 측정 시작
# start_time = time.time()

# # 4bit 양자화(QLoRA)로 모델 로드
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     load_in_4bit=True,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
# )
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# # LoRA 설정
# peft_config = LoraConfig(
#     r=8,
#     lora_alpha=32,
#     target_modules=["q_proj", "v_proj"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type=TaskType.CAUSAL_LM
# )
# model = get_peft_model(model, peft_config)

# # 실행 시간 측정 끝 및 출력
# elapsed = time.time() - start_time
# print(f"모델 로딩과 LoRA 적용까지 걸린 시간: {elapsed:.2f}초")


In [20]:
# 토크나이저에 패딩 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    tokens = tokenizer(
        prompt,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors=None  # 배치 차원 제거
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized = dataset.map(tokenize, remove_columns=dataset.column_names)

Map:   0%|          | 0/322 [00:00<?, ? examples/s]

In [21]:
%%time

training_args = TrainingArguments(
    output_dir="./hearth_llm",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=10,
    save_steps=100,
    max_steps=500,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=1,
    report_to=[],  # wandb 비활성화
    remove_unused_columns=False,  # 추가
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    tokenizer=tokenizer
)

trainer.train()

<timed exec>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:457: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
10,6.179300
20,6.047600
30,6.042200
40,6.116200
50,6.126700
60,6.103100
70,6.111600
80,6.022100
90,6.105600
100,6.104300


CPU times: user 35min 18s, sys: 18min 37s, total: 53min 55s
Wall time: 54min 18s


TrainOutput(global_step=500, training_loss=5.981761924743652, metrics={'train_runtime': 3257.6909, 'train_samples_per_second': 0.614, 'train_steps_per_second': 0.153, 'total_flos': 4.344658059381965e+16, 'train_loss': 5.981761924743652, 'epoch': 6.173913043478261})

In [23]:
# 파인튜닝된 모델 저장
model.save_pretrained("hearth_llm_model")
tokenizer.save_pretrained("hearth_llm_model")

print("파인튜닝된 모델 저장 완료!")
print("저장 위치: ./hearth_llm_model/")

파인튜닝된 모델 저장 완료!
저장 위치: ./hearth_llm_model/


In [24]:
from transformers import pipeline

# 파인튜닝된 모델로 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model="hearth_llm_model",
    tokenizer="hearth_llm_model",
    device_map="auto"
)

# 테스트 프롬프트들
test_prompts = [
    "### Instruction:\n간단한 자기소개를 해줘\n\n### Response:\n",
    "### Instruction:\n오늘 기분이 우울해\n\n### Response:\n",
    "### Instruction:\n프로그래밍에 대해 설명해줘\n\n### Response:\n"
]

# 각 프롬프트 테스트
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n--- 테스트 {i} ---")
    print(f"프롬프트: {prompt}")

    result = pipe(prompt, max_new_tokens=100, temperature=0.7)
    response = result[0]['generated_text']

    print(f"응답: {response}")
    print("-" * 50)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.layers.22.self_attn.q_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.layers.22.self_attn.q_proj.lora_B.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.layers.22.self_attn.v_proj.lora_A.default.weight: 

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.layers.4.self_attn.q_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.layers.4.self_attn.q_proj.lora_B.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.layers.4.self_attn.v_proj.lora_A.default.weight: cop

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.layers.1.self_attn.q_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.layers.1.self_attn.q_proj.lora_B.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.layers.1.self_attn.v_proj.lora_A.default.weight: cop

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

ValueError: Could not load model hearth_llm_model with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.mistral.modeling_mistral.MistralForCausalLM'>, <class 'transformers.models.mistral.modeling_tf_mistral.TFMistralForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py", line 292, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py", line 600, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transf

In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# 1. 기본 모델과 토크나이저 로드
base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)

# 2. LoRA 어댑터 로드
model = PeftModel.from_pretrained(base_model, "hearth_llm_model")

# 3. 토크나이저 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ 파인튜닝된 모델 로드 완료!")

# 4. 테스트
def generate_response(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# 5. 테스트 프롬프트들
test_prompts = [
    "### Instruction:\n간단한 자기소개를 해줘\n\n### Response:\n",
    "### Instruction:\n오늘 기분이 우울해\n\n### Response:\n",
    "### Instruction:\n프로그래밍에 대해 설명해줘\n\n### Response:\n"
]

# 6. 각 프롬프트 테스트
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n--- 테스트 {i} ---")
    print(f"프롬프트: {prompt}")

    try:
        response = generate_response(prompt)
        print(f"응답: {response}")
    except Exception as e:
        print(f"오류: {e}")

    print("-" * 50)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

# 1. 더 가벼운 설정으로 기본 모델 로드
base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# 4bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# GPU 메모리 최적화 설정
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    offload_folder="offload"  # 디스크 오프로드 허용
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)

# 2. LoRA 어댑터 로드
model = PeftModel.from_pretrained(base_model, "hearth_llm_model")

# 3. 토크나이저 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ 파인튜닝된 모델 로드 완료!")

# 4. 간단한 테스트
def simple_test():
    prompt = "### Instruction:\n안녕하세요! 간단한 인사말을 해주세요.\n\n### Response:\n"

    print(f"테스트 프롬프트: {prompt}")

    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                inputs["input_ids"],
                max_new_tokens=50,  # 더 짧게
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"응답: {response}")
        print("✅ 테스트 성공!")

    except Exception as e:
        print(f"❌ 테스트 실패: {e}")

# 5. 테스트 실행
simple_test()

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [28]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

# 더 작은 모델로 테스트 (선택사항)
from transformers import pipeline

# DialoGPT-medium (345M 파라미터)로 테스트
test_model = "microsoft/DialoGPT-medium"
test_tokenizer = AutoTokenizer.from_pretrained(test_model)
test_model = AutoModelForCausalLM.from_pretrained(test_model)

# 간단한 테스트
test_prompt = "안녕하세요!"
inputs = test_tokenizer.encode(test_prompt + test_tokenizer.eos_token, return_tensors='pt')

with torch.no_grad():
    outputs = test_model.generate(inputs, max_length=100, pad_token_id=test_tokenizer.eos_token_id)

response = test_tokenizer.decode(outputs[:, inputs.shape[-1]:][0], skip_special_tokens=True)
print(f"테스트 응답: {response}")

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


테스트 응답: I'm not sure if you're being serious or not, but I'm pretty sure that's a Korean word.


In [29]:
# LoRA 어댑터만 저장 (이미 완료됨)
print("LoRA 어댑터 저장 위치: ./hearth_llm_model/")

# 파일 목록 확인
import os
print("저장된 파일들:")
for file in os.listdir("hearth_llm_model"):
    print(f"  - {file}")

LoRA 어댑터 저장 위치: ./hearth_llm_model/
저장된 파일들:
  - tokenizer.model
  - adapter_model.safetensors
  - adapter_config.json
  - chat_template.jinja
  - tokenizer.json
  - README.md
  - tokenizer_config.json
  - special_tokens_map.json


In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

# 더 작은 모델로 테스트 (선택사항)
from transformers import pipeline

# DialoGPT-medium (345M 파라미터)로 테스트
test_model = "microsoft/DialoGPT-medium"
test_tokenizer = AutoTokenizer.from_pretrained(test_model)
test_model = AutoModelForCausalLM.from_pretrained(test_model)

# 간단한 테스트
test_prompt = "[INST] 안녕하세요! 자기소개 해줘. [/INST]"
inputs = test_tokenizer.encode(test_prompt + test_tokenizer.eos_token, return_tensors='pt')
attention_mask = torch.ones(inputs.shape, dtype=torch.long)

with torch.no_grad():
    outputs = test_model.generate(
        inputs,
        attention_mask=attention_mask,
        max_length=100,
        pad_token_id=test_tokenizer.eos_token_id
    )

response = test_tokenizer.decode(outputs[:, inputs.shape[-1]:][0], skip_special_tokens=True)
print(f"테스트 응답: {response}")

테스트 응답: I'm not sure if you're being serious or not, but I'm pretty sure that's a joke.


In [15]:
# LoRA 어댑터만 저장 (이미 완료됨)
print("LoRA 어댑터 저장 위치: ./hearth_llm_model/")

# 파일 목록 확인
import os
print("저장된 파일들:")
for file in os.listdir("hearth_llm_model"):
    print(f"  - {file}")

LoRA 어댑터 저장 위치: ./hearth_llm_model/
저장된 파일들:
  - tokenizer.model
  - adapter_model.safetensors
  - adapter_config.json
  - chat_template.jinja
  - tokenizer.json
  - README.md
  - tokenizer_config.json
  - special_tokens_map.json


In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 한국어 모델로 변경
test_model_name = "EleutherAI/polyglot-ko-1.3b"
test_model = AutoModelForCausalLM.from_pretrained(test_model_name).to("cuda")
test_tokenizer = AutoTokenizer.from_pretrained(test_model_name)

# 간단한 테스트
test_prompt = "오늘 기분이 어때?"
inputs = test_tokenizer.encode(test_prompt, return_tensors='pt').to("cuda")
attention_mask = torch.ones(inputs.shape, dtype=torch.long).to("cuda")

with torch.no_grad():
    outputs = test_model.generate(
        inputs,
        attention_mask=attention_mask,
        max_length=100,
        pad_token_id=test_tokenizer.eos_token_id,
        )

response = test_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"테스트 응답: {response}")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 42.12 MiB is free. Process 574936 has 14.70 GiB memory in use. Of the allocated memory 14.47 GiB is allocated by PyTorch, and 109.48 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [12]:
# LoRA 어댑터만 저장 (이미 완료됨)
print("LoRA 어댑터 저장 위치: ./hearth_llm_model_kor/")

# 파일 목록 확인
import os
print("저장된 파일들:")
for file in os.listdir("hearth_llm_model_kor"):
    print(f"  - {file}")

LoRA 어댑터 저장 위치: ./hearth_llm_model_kor/
저장된 파일들:


In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# 1. 기본 모델과 토크나이저 로드
base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# 2. LoRA 어댑터가 적용된 모델 로드
model_to_merge = PeftModel.from_pretrained(base_model, "hearth_llm_model")

# 3. LoRA 어댑터 병합
merged_model = model_to_merge.merge_and_unload()

print("✅ LoRA 어댑터 병합 완료!")

# 4. 테스트
def generate_response(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(merged_model.device)

    with torch.no_grad():
        outputs = merged_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# 5. 테스트 프롬프트들
test_prompts = [
    "### Instruction:\n간단한 자기소개를 해줘\n\n### Response:\n",
    "### Instruction:\n오늘 기분이 우울해\n\n### Response:\n",
    "### Instruction:\n프로그래밍에 대해 설명해줘\n\n### Response:\n"
]

# 6. 각 프롬프트 테스트
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n--- 테스트 {i} ---")
    print(f"프롬프트: {prompt}")

    try:
        response = generate_response(prompt)
        print(f"응답: {response}")
    except Exception as e:
        print(f"오류: {e}")

    print("-" * 50)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

ValueError: We need an `offload_dir` to dispatch this model according to this `device_map`, the following submodules need to be offloaded: base_model.model.model.layers.16, base_model.model.model.layers.17, base_model.model.model.layers.18, base_model.model.model.layers.19, base_model.model.model.layers.20, base_model.model.model.layers.21, base_model.model.model.layers.22, base_model.model.model.layers.23, base_model.model.model.layers.24, base_model.model.model.layers.25, base_model.model.model.layers.26, base_model.model.model.layers.27, base_model.model.model.layers.28, base_model.model.model.layers.29, base_model.model.model.layers.30, base_model.model.model.layers.31, base_model.model.model.norm, base_model.model.model.rotary_emb, base_model.model.lm_head.